# 余弦相似度

In [ ]:
- 余弦相似度适合用于自然语言文本、图像和高维场景
- 欧式距离适合用于数值数据和低维数据

## 准备

In [1]:
import faiss
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer

## 最基本示例

In [2]:
# 创建一些示例向量
vecs = np.random.random((10, 64)).astype('float32')

# 对向量进行归一化
vecs /= np.linalg.norm(vecs, axis=1)[:, np.newaxis]

# 创建一个FlatIndex
index = faiss.IndexFlatIP(64)  # 使用内积作为相似度度量

# 向索引中添加向量
index.add(vecs)

# 查询向量
query_vec = np.random.random((1, 64)).astype('float32')
query_vec /= np.linalg.norm(query_vec)

# 进行查询
k = 5  # 查询前5个最相似的向量
distances, indices = index.search(query_vec, k)

print("最相似的向量索引：", indices)
print("相似度度量值：", distances)

最相似的向量索引： [[8 7 1 0 4]]
相似度度量值： [[0.8394477  0.79711246 0.7828023  0.77742726 0.76630914]]


## 给猫吃点啥

In [3]:
data = [['六分仪（Sextant）是一种用于测量天体角度的仪器，常用于航海和天文导航。', '工具'],
['烤白薯是一道美味健康的小吃，下面是简单的制作步骤', '菜谱'],
['老鼠是一种常见的小型哺乳动物，属于鼠科。', '动物'],
['比瑞吉 优选系列 添加深海鱼油 成猫猫粮2kg', '电商'],
['奶酪是一种以牛、羊、山羊或其他动物的乳汁为原料，经过发酵和加工制作而成的食品。', '食品'],
['凯锐思 幼猫猫粮奶糕全价幼猫粮天然粮 【尝鲜装】幼猫粮500g','电商'],
['黑洞是宇宙中一种极为神秘的天体，它是一种极度密集的天体，其引力非常强大，甚至连光都无法逃离其吸引。','天体物理'],
['"阿拉斯加的鳄鱼" 这个短语通常是一种幽默或谐音，用来形容一个不存在的事物或情况。','修辞'],
['潭中鱼可百许头，皆若空游无所依。','古代文学'],
['武松是中国古典小说《水浒传》中的一个著名人物，他是108位好汉中的一员，也是四大头领之一。','古代文学']]
df = pd.DataFrame(data, columns = ['文本', '分类'])

print('\n数据:')
display(df)

text = df['文本']


数据:


,文本,分类
0,六分仪（Sextant）是一种用于测量天体角度的仪器，常用于航海和天文导航。,工具
1,烤白薯是一道美味健康的小吃，下面是简单的制作步骤,菜谱
2,老鼠是一种常见的小型哺乳动物，属于鼠科。,动物
3,比瑞吉 优选系列 添加深海鱼油 成猫猫粮2kg,电商
4,奶酪是一种以牛、羊、山羊或其他动物的乳汁为原料，经过发酵和加工制作而成的食品。,食品
5,凯锐思 幼猫猫粮奶糕全价幼猫粮天然粮 【尝鲜装】幼猫粮500g,电商
6,黑洞是宇宙中一种极为神秘的天体，它是一种极度密集的天体，其引力非常强大，甚至连光都无法逃离其吸引。,天体物理
7,"""阿拉斯加的鳄鱼"" 这个短语通常是一种幽默或谐音，用来形容一个不存在的事物或情况。",修辞
8,潭中鱼可百许头，皆若空游无所依。,古代文学
9,武松是中国古典小说《水浒传》中的一个著名人物，他是108位好汉中的一员，也是四大头领之一。,古代文学


In [4]:
%%time

encoder = SentenceTransformer("/models/bge-large-zh-v1.5")
vectors = encoder.encode(text)
vector_dimension = vectors.shape[1]

vector_dimension

CPU times: user 1.57 s, sys: 671 ms, total: 2.24 s
Wall time: 1.96 s


1024

In [5]:
vectors

array([[ 0.02792057, -0.03714846, -0.01159202, ..., -0.01725691,
        -0.00189179, -0.0196649 ],
       [-0.00108438, -0.04289852,  0.03477975, ...,  0.01018529,
         0.02182655, -0.00230852],
       [-0.00641309, -0.02665936,  0.0109663 , ...,  0.01449825,
         0.02765447,  0.00925979],
       ...,
       [-0.04652889, -0.0133617 , -0.00904603, ..., -0.00231135,
        -0.00262135,  0.03224764],
       [ 0.02496061,  0.0328386 , -0.01557181, ..., -0.03958136,
        -0.03401508,  0.04865717],
       [-0.00382815, -0.03929801,  0.05309108, ..., -0.01528487,
         0.01552107,  0.01694353]], dtype=float32)

In [6]:
index = faiss.IndexFlatIP(vector_dimension)
index.add(vectors)

In [7]:
search_text = '给小猫吃点啥?'
search_text

'给小猫吃点啥?'

In [8]:
search_vector = encoder.encode(search_text)
_vector = np.array([search_vector])

k = index.ntotal
distances, ann = index.search(_vector, k=k)

results = pd.DataFrame({'distances': distances[0], 'ann': ann[0]})
merge=pd.merge(results,df,left_on='ann',right_index=True)

print('\n相似度排序')
display(merge)


相似度排序


,distances,ann,文本,分类
0,0.426950,5,凯锐思 幼猫猫粮奶糕全价幼猫粮天然粮 【尝鲜装】幼猫粮500g,电商
1,0.321440,3,比瑞吉 优选系列 添加深海鱼油 成猫猫粮2kg,电商
2,0.311746,2,老鼠是一种常见的小型哺乳动物，属于鼠科。,动物
3,0.300995,1,烤白薯是一道美味健康的小吃，下面是简单的制作步骤,菜谱
4,0.270809,4,奶酪是一种以牛、羊、山羊或其他动物的乳汁为原料，经过发酵和加工制作而成的食品。,食品
5,0.172551,8,潭中鱼可百许头，皆若空游无所依。,古代文学
6,0.122142,6,黑洞是宇宙中一种极为神秘的天体，它是一种极度密集的天体，其引力非常强大，甚至连光都无法逃离其吸引。,天体物理
7,0.117711,7,"""阿拉斯加的鳄鱼"" 这个短语通常是一种幽默或谐音，用来形容一个不存在的事物或情况。",修辞
8,0.068569,0,六分仪（Sextant）是一种用于测量天体角度的仪器，常用于航海和天文导航。,工具
9,0.046192,9,武松是中国古典小说《水浒传》中的一个著名人物，他是108位好汉中的一员，也是四大头领之一。,古代文学
